In [2]:
# Author: Alden Sahi
# Project Name: EarningsReportJunkie
# Project Description:
  # User specifies stock
  # Returns Predicted Capital Gain using disparity of stock price that predicted earnings
  # suggests vs reported earnings

# Model needs: predicted stock price at the data of earnings, predicted earnings, and the last pe ratio from last earnings
# THis means wee nee

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

In [75]:
# User Input
ticker='IBM'
alpha_key = 'TQCF34ZIGZ7OTPLO'
train_data_start = '1996-03-31'
train_data_end = '2024-12-31'
days_till_money = 4
threshold = 0.05 # percentage gain after day (greenflag)

In [3]:
# Hypothesis: Buying a long period of time stock over a period of time missies
# the power of compound interest

# Datafrane Columns: A bunch of stuff from the earnings report and a vwma

# My idea is to create buy signals with great earnings and 
# sell signals once the vwma egts crossed

# This strategy will output returns year after year

In [4]:
# Proof of Concept
# Show average earnings after 1 5 and 10 years of a company
#S Show earnigns after 1 4 and 10 years while buyings and sellings

## Getting Data

In [5]:
# call api for earnings report data

url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={alpha_key}'
r = requests.get(url)
data = r.json()

# Check if the API call was successful
if 'quarterlyEarnings' not in data:
    raise ValueError(f"Failed to fetch earnings data for {ticker}. Check your API key or ticker symbol.")

# congolmerate data to df
earnings_data = pd.DataFrame(data['quarterlyEarnings'])

In [6]:
# api for cash flow

url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={alpha_key}]'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'operatingCashflow': '13931000000', 'paymentsForOperatingActivities': '2645000000', 'proceedsFromOperatingActivities': 'None', 'changeInOperatingLiabilities': '65000000', 'changeInOperatingAssets': '-1115000000', 'depreciationDepletionAndAmortization': '4396000000', 'capitalExpenditures': '1245000000', 'changeInReceivables': '-725000000', 'changeInInventory': '-390000000', 'profitLoss': '7523000000', 'cashflowFromInvestment': '-7070000000', 'cashflowFromFinancing': '-1769000000', 'proceedsFromRepaymentsOfShortTermDebt': '-7000000', 'paymentsForRepurchaseOfCommonStock': 'None', 'paymentsForRepurchaseOfEquity': 'None', 'paymentsForRepurchaseOfPreferredStock': 'None', 'dividendPayout': '6040000000', 'dividendPayoutCommonStock': '6040000000', 'dividendPayoutPreferredStock': 'None', 'proceedsFromIssuanceOfCommonStock': 'None', 'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet': '958600

In [7]:
# make into dataframe
cash_flow_data = pd.DataFrame(data['quarterlyReports'])
cash_flow_data

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome
0,2024-09-30,USD,-745000000,None,None,None,None,1268000000,424000000,None,...,1543000000,None,None,0,None,-189000000,None,None,None,-330000000
1,2024-03-31,USD,4168000000,None,None,None,None,1131000000,239000000,None,...,1522000000,None,None,5486000000,None,-199000000,None,None,None,1605000000
2,2023-12-31,USD,4463000000,None,None,None,None,1152000000,300000000,None,...,1518000000,None,None,0,None,-64000000,None,None,None,3288000000
3,2023-09-30,USD,3056000000,None,None,None,None,1093000000,281000000,None,...,1515000000,None,None,154000000,None,0,None,None,None,1704000000
4,2023-06-30,USD,2638000000,None,None,None,None,1077000000,364000000,None,...,1510000000,None,None,0,None,0,None,-85000000,None,1583000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2009-06-30,USD,4741000000,None,None,None,None,1242000000,863000000,None,...,732000000,None,None,1056000000,None,-1671000000,None,-616000000,235000000,3103000000
61,2009-03-31,USD,4386000000,None,None,None,None,1229000000,760000000,None,...,675000000,None,None,913000000,None,-1765000000,None,-447000000,-202000000,2295000000
62,2008-12-31,USD,6621000000,None,None,None,None,1321000000,1381000000,None,...,669000000,None,None,5578000000,None,-740000000,None,2986000000,116000000,4427000000
63,2008-09-30,USD,3738000000,None,None,None,None,1379000000,1126000000,None,...,677000000,None,None,1438000000,None,-2674000000,None,129000000,-233000000,2823000000


In [8]:
# api for balance sheet

url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={alpha_key}'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'totalAssets': '135241000000', 'totalCurrentAssets': '32908000000', 'cashAndCashEquivalentsAtCarryingValue': '13068000000', 'cashAndShortTermInvestments': '13068000000', 'inventory': '1161000000', 'currentNetReceivables': '7725000000', 'totalNonCurrentAssets': '101302000000', 'propertyPlantEquipment': '-472000000', 'accumulatedDepreciationAmortizationPPE': 'None', 'intangibleAssets': '71214000000', 'intangibleAssetsExcludingGoodwill': '11036000000', 'goodwill': '60178000000', 'investments': '125000000', 'longTermInvestments': '125000000', 'shortTermInvestments': '373000000', 'otherCurrentAssets': '10587000000', 'otherNonCurrentAssets': 'None', 'totalLiabilities': '112628000000', 'totalCurrentLiabilities': '34122000000', 'currentAccountsPayable': '4132000000', 'deferredRevenue': '16984000000', 'currentDebt': '12851000000', 'shortTermDebt': '6426000000', 'totalNonCurrentLiabilities': '870720

In [9]:
# make into dataframe
balance_sheet_data = pd.DataFrame(data['quarterlyReports'])
balance_sheet_data 

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2024-09-30,USD,134339000000,30543000000,13197000000,13197000000,1367000000,6203000000,96174000000,5614000000,...,3590000000,52980000000,82312000000,5934000000,11186000000,24448000000,169935000000,149789000000,61013000000,923600000
1,2024-03-31,USD,137169000000,36663000000,14603000000,14603000000,1212000000,6676000000,93235000000,5594000000,...,5469000000,54033000000,86750000000,6009000000,10914000000,23261000000,169759000000,151362000000,60145000000,917200000
2,2023-12-31,USD,135241000000,32908000000,13068000000,13068000000,1161000000,7725000000,101302000000,-472000000,...,6304000000,50121000000,120630000000,6203000000,11475000000,22533000000,169624000000,151276000000,59643000000,915013646
3,2023-09-30,USD,129321000000,27705000000,7257000000,7257000000,1399000000,6039000000,100035000000,5369000000,...,6400000000,48828000000,84575000000,8126000000,12081000000,23081000000,169640000000,149506000000,59313000000,912800000
4,2023-06-30,USD,132213000000,34458000000,9394000000,9394000000,1501000000,6368000000,96170000000,5443000000,...,6780000000,50691000000,87088000000,8442000000,12103000000,22201000000,169581000000,149318000000,58963000000,909855943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2010-06-30,USD,103420000000,44895000000,10325000000,12241000000,2595000000,9051000000,49339000000,13534000000,...,None,None,26650000000,4539000000,8295000000,21059000000,89276000000,85323000000,43522000000,1278600000
57,2010-03-31,USD,105208000000,45697000000,12472000000,13977000000,2524000000,9324000000,49968000000,13841000000,...,None,None,26319000000,5285000000,8506000000,22033000000,85238000000,82783000000,42665000000,1301200000
58,2009-12-31,USD,109022000000,48935000000,12183000000,13974000000,2494000000,10736000000,49443000000,14165000000,...,2222000000,None,26100000000,5223000000,8819000000,22637000000,81243000000,80900000000,41810000000,1318329640
59,2009-09-30,USD,103675000000,43446000000,9532000000,11512000000,2546000000,9426000000,50000000000,14189000000,...,None,None,25468000000,5704000000,9863000000,18396000000,78411000000,76812000000,40482000000,1319900000


In [10]:
# Income Statement APi

url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={alpha_key}'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'grossProfit': '34300000000', 'totalRevenue': '61860000000', 'costOfRevenue': '27560000000', 'costofGoodsAndServicesSold': '245000000', 'operatingIncome': '6979000000', 'sellingGeneralAndAdministrative': '19003000000', 'researchAndDevelopment': '6775000000', 'operatingExpenses': '27321000000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-1607000000', 'interestIncome': '591000000', 'interestExpense': '1607000000', 'nonInterestIncome': '-77000000', 'otherNonOperatingIncome': '266000000', 'depreciation': '2109000000', 'depreciationAndAmortization': '2287000000', 'incomeBeforeTax': '8678000000', 'incomeTaxExpense': '1176000000', 'interestAndDebtExpense': '1607000000', 'netIncomeFromContinuingOperations': '7514000000', 'comprehensiveIncomeNetOfTax': '5481000000', 'ebit': '10285000000', 'ebitda': '12572000000', 'netIncome': '7502000000'}, {'fiscalDateEnding': '2022-12-31', 'reportedCurr

In [11]:
# make into dataframe
income_stmt_data = pd.DataFrame(data['quarterlyReports'])
income_stmt_data 

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,2024-09-30,USD,8420000000,14968000000,6548000000,12000000,1364000000,4911000000,1876000000,7056000000,...,None,1268000000,-815000000,-485000000,None,-317000000,1570000000,-386000000,895000000,-330000000
1,2024-03-31,USD,7742000000,14462000000,6719000000,28000000,85000000,4974000000,1796000000,7657000000,...,533000000,598000000,1103000000,-502000000,432000000,1575000000,1878000000,1535000000,2133000000,1605000000
2,2023-12-31,USD,10267000000,17381000000,7114000000,112000000,3876000000,4791000000,1748000000,6391000000,...,541000000,611000000,3762000000,474000000,405000000,3285000000,624000000,4167000000,4778000000,3288000000
3,2023-09-30,USD,8023000000,14752000000,6729000000,42000000,1994000000,4458000000,1685000000,6029000000,...,521000000,572000000,1863000000,159000000,412000000,1714000000,2105000000,2275000000,2847000000,1704000000
4,2023-06-30,USD,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,...,520000000,557000000,2002000000,419000000,423000000,1581000000,1863000000,2425000000,2982000000,1583000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2010-06-30,USD,10809000000,23724000000,12915000000,12641000000,3858000000,5061000000,1475000000,7041000000,...,899000000,284000000,4575000000,1190000000,90000000,3385000000,6438000000,4665000000,4949000000,3385000000
57,2010-03-31,USD,9976000000,22857000000,12880000000,12608000000,2504000000,5677000000,1509000000,7554000000,...,924000000,286000000,3515000000,914000000,82000000,2601000000,2601000000,3597000000,3883000000,2601000000
58,2009-12-31,USD,13145000000,27230000000,14085000000,13784000000,5827000000,5560000000,1460000000,7399000000,...,967000000,298000000,6381000000,1568000000,81000000,4813000000,21253000000,6462000000,6760000000,4813000000
59,2009-09-30,USD,10627000000,23566000000,12938000000,12637000000,4168000000,5013000000,1446000000,6543000000,...,953000000,305000000,4373000000,1159000000,84000000,3214000000,3214000000,4457000000,4762000000,3214000000


In [200]:
# prices api
prices = yf.download(ticker, start=train_data_start, end=train_data_end)

[*********************100%***********************]  1 of 1 completed


## Merging and Filtering Data

In [ ]:
total_report = earnings_data.merge(cash_flow_data, on='fiscalDateEnding') \
                           .merge(income_stmt_data, on='fiscalDateEnding') \
                           .merge(balance_sheet_data, on='fiscalDateEnding')

In [201]:
#joins into single index (was multtIndex)
prices.columns = ['_'.join(col) for col in prices.columns]


In [202]:
# Create a new column from the index so we can merge easy with total_report
prices['fiscalDateEnding'] = prices.index

# Convert 'fiscalDateEnding' to datetime64[ns]
total_report['fiscalDateEnding'] = pd.to_datetime(total_report['fiscalDateEnding'])

In [162]:
## MORE ACCURATE TBT

# makes new row with dates for


# Create a new column in prices for output
#prices['Aligned_Price'] = None

# for every row in total_report
#for idx, row in total_report.iterrows():
    # if the corresponding rows' report time is premarket
    #if total_report['reportTime_x'][idx] == 'pre-market':
        # save open price to the prices with that idx
       # prices.loc[date] = 1
    # if the corresponding rows' report time is postmarket
    #if total_report['reportTime_x'][idx] == 'poste-market':
        # save the open price of the next day
          #prices.loc[date] = 0
        

    

In [203]:
# sort both datasets to ensure they are in chronological order
earnings_data = earnings_data.sort_index()
prices = prices.sort_index()

In [206]:
prices = prices[[f'Open_{ticker}','fiscalDateEnding']]
prices

,Open_IBM,fiscalDateEnding
Date,,
1996-04-01,13.725689,1996-04-01
1996-04-02,13.587981,1996-04-02
1996-04-03,14.383678,1996-04-03
1996-04-04,14.674410,1996-04-04
1996-04-08,14.154145,1996-04-08
...,...,...
2024-12-23,221.335424,2024-12-23
2024-12-24,220.798999,2024-12-24
2024-12-26,221.832112,2024-12-26


In [212]:
# prices 
prices[f'Next_Day_{ticker}'] = prices[f'Open_{ticker}'].shift(-1)

In [213]:
prices

,Open_IBM,fiscalDateEnding,Next_Day_IBM
Date,,,
1996-04-01,13.725689,1996-04-01,13.587981
1996-04-02,13.587981,1996-04-02,14.383678
1996-04-03,14.383678,1996-04-03,14.674410
1996-04-04,14.674410,1996-04-04,14.154145
1996-04-08,14.154145,1996-04-08,14.659099
...,...,...,...
2024-12-23,221.335424,2024-12-23,220.798999
2024-12-24,220.798999,2024-12-24,221.832112
2024-12-26,221.832112,2024-12-26,221.663236


In [214]:
total_report = total_report.merge(prices,on=['fiscalDateEnding'])

In [215]:
total_report

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,reportedCurrency_x,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_x,reportedCurrency_y,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Open_IBM,Next_Day_IBM
0,2024-09-30,2024-10-23,2.3,2.23,0.07,3.139,post-market,USD,-745000000,None,None,None,None,1268000000,424000000,None,None,-317000000,None,-2765000000,0,None,None,None,1543000000,1543000000,None,None,0,None,-189000000,None,None,None,-330000000,USD,8420000000,14968000000,6548000000,12000000,1364000000,4911000000,1876000000,7056000000,None,-259000000,170000000,429000000,-466000000,422000000,None,1268000000,-815000000,-485000000,None,-317000000,1570000000,-386000000,895000000,-330000000,USD,134339000000,30543000000,13197000000,13197000000,1367000000,6203000000,96174000000,5614000000,None,72182000000,11090000000,61092000000,None,None,505000000,1836000000,None,109809000000,28853000000,3274000000,16548000000,7189000000,3599000000,80955000000,3547000000,55308000000,3590000000,52980000000,82312000000,5934000000,11186000000,24448000000,169935000000,149789000000,61013000000,923600000,217.475775,217.456058
1,2023-06-30,2023-07-19,2.18,2.01,0.17,8.4577,post-market,USD,2638000000,None,None,None,None,1077000000,364000000,None,None,1581000000,7000000,-2730000000,0,0,0,None,1510000000,1510000000,None,None,0,None,0,None,-85000000,None,1583000000,USD,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,None,-423000000,201000000,423000000,163000000,39000000,520000000,557000000,2002000000,419000000,423000000,1581000000,1863000000,2425000000,2982000000,1583000000,USD,132213000000,34458000000,9394000000,9394000000,1501000000,6368000000,96170000000,5443000000,13145000000,66881000000,10496000000,56385000000,None,1585000000,6904000000,2731000000,None,109942000000,32513000000,3732000000,15976000000,13565000000,6785000000,77429000000,2828000000,54030000000,6780000000,50691000000,87088000000,8442000000,12103000000,22201000000,169581000000,149318000000,58963000000,909855943,126.229859,125.039629
2,2023-03-31,2023-04-19,1.36,1.26,0.1,7.9365,po

In [171]:
total_report

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,reportedCurrency_x,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_x,reportedCurrency_y,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Close_IBM,High_IBM,Low_IBM,Open_IBM,Volume_IBM,Next_Day_IBM,Buy Signal
0,2024-09-30,2024-10-23,2.3,2.23,0.07,3.139,post-market,USD,-745000000,None,None,None,None,1268000000,424000000,None,None,-317000000,None,-2765000000,0,None,None,None,1543000000,1543000000,None,None,0,None,-189000000,None,None,None,-330000000,USD,8420000000,14968000000,6548000000,12000000,1364000000,4911000000,1876000000,7056000000,None,-259000000,170000000,429000000,-466000000,422000000,None,1268000000,-815000000,-485000000,None,-317000000,1570000000,-386000000,895000000,-330000000,USD,134339000000,30543000000,13197000000,13197000000,1367000000,6203000000,96174000000,5614000000,None,72182000000,11090000000,61092000000,None,None,505000000,1836000000,None,109809000000,28853000000,3274000000,16548000000,7189000000,3599000000,80955000000,3547000000,55308000000,3590000000,52980000000,82312000000,5934000000,11186000000,24448000000,169935000000,149789000000,61013000000,923600000,217.899597,218.136150,215.869234,217.475775,3543400,219.791970,0
1,2023-06-30,2023-07-19,2.18,2.01,0.17,8.4577,post-market,USD,2638000000,None,None,None,None,1077000000,364000000,None,None,1581000000,7000000,-2730000000,0,0,0,None,1510000000,1510000000,None,None,0,None,0,None,-85000000,None,1583000000,USD,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,None,-423000000,201000000,423000000,163000000,39000000,520000000,557000000,2002000000,419000000,423000000,1581000000,1863000000,2425000000,2982000000,1583000000,USD,132213000000,34458000000,9394000000,9394000000,1501000000,6368000000,96170000000,5443000000,13145000000,66881000000,10496000000,56385000000,None,1585000000,6904000000,2731000000,None,109942000000,32513000000,3732000000,15976000000,13565000000,6785000000,77429000000,2828000000,54030000000,6780000000,50691000000,87088000000,8442000000,12103000000,22201000000,169581000000,149318000000

In [217]:
# converts columns to floats for computation, invalid parsing is set to Nan
total_report['reportedEPS'] = pd.to_numeric(total_report['reportedEPS'], errors='coerce')
total_report['estimatedEPS'] = pd.to_numeric(total_report['estimatedEPS'], errors='coerce')
total_report[f'Open_{ticker}'] = pd.to_numeric(total_report[f'Open_{ticker}'], errors='coerce') 


# calculate predicted and reported pe ratios
total_report['estimatedPE'] = total_report[f'Open_{ticker}'] / total_report['estimatedEPS']
total_report['reportedPE'] = total_report[f'Open_{ticker}'] / total_report['reportedEPS']

# calculate predicted vs reported price
total_report['estimatedPrice'] = total_report['estimatedEPS'] * total_report['estimatedPE']
total_report['reportedPrice'] = total_report['reportedEPS'] * total_report['estimatedPE']

# calcuclate difference between estimated and reported
total_report['priceDiscrepancy'] = total_report['reportedPrice'] - total_report['estimatedPrice']  


# calculates change in price after 7 days after earnings release
#total_report['priceChange'] = total_report[ticker].diff(7)
total_report


,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,reportedCurrency_x,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_x,reportedCurrency_y,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Open_IBM,Next_Day_IBM,estimatedPE,reportedPE,estimatedPrice,reportedPrice,priceDiscrepancy
0,2024-09-30,2024-10-23,2.30,2.23,0.07,3.139,post-market,USD,-745000000,None,None,None,None,1268000000,424000000,None,None,-317000000,None,-2765000000,0,None,None,None,1543000000,1543000000,None,None,0,None,-189000000,None,None,None,-330000000,USD,8420000000,14968000000,6548000000,12000000,1364000000,4911000000,1876000000,7056000000,None,-259000000,170000000,429000000,-466000000,422000000,None,1268000000,-815000000,-485000000,None,-317000000,1570000000,-386000000,895000000,-330000000,USD,134339000000,30543000000,13197000000,13197000000,1367000000,6203000000,96174000000,5614000000,None,72182000000,11090000000,61092000000,None,None,505000000,1836000000,None,109809000000,28853000000,3274000000,16548000000,7189000000,3599000000,80955000000,3547000000,55308000000,3590000000,52980000000,82312000000,5934000000,11186000000,24448000000,169935000000,149789000000,61013000000,923600000,217.475775,217.456058,97.522769,94.554685,217.475775,224.302369,6.826594
1,2023-06-30,2023-07-19,2.18,2.01,0.17,8.4577,post-market,USD,2638000000,None,None,None,None,1077000000,364000000,None,None,1581000000,7000000,-2730000000,0,0,0,None,1510000000,1510000000,None,None,0,None,0,None,-85000000,None,1583000000,USD,8501000000,15475000000,6974000000,61000000,1984000000,4900000000,1687000000,6517000000,None,-423000000,201000000,423000000,163000000,39000000,520000000,557000000,2002000000,419000000,423000000,1581000000,1863000000,2425000000,2982000000,1583000000,USD,132213000000,34458000000,9394000000,9394000000,1501000000,6368000000,96170000000,5443000000,13145000000,66881000000,10496000000,56385000000,None,1585000000,6904000000,2731000000,None,109942000000,32513000000,3732000000,15976000000,13565000000,6785000000,77429000000,2828000000,54030000000,6780000000,50691000000,87088000000,8442000000,12103000000,22201000

In [ ]:
# drops columns that are uneeded
# todo if it has none at all

total_report = total_report.drop(columns=[
    'surprise', 'surprisePercentage', 'reportedEPS', 'estimatedEPS', 'reportTime',
    'estimatedPE', 'reportedPE', 'estimatedPrice', 'reportedPrice', f'Open_{ticker}',
    'reportedCurrency_x', 'paymentsForOperatingActivities', 'proceedsFromOperatingActivities',
    'changeInOperatingLiabilities', 'changeInOperatingAssets', 'changeInReceivables',
    'changeInInventory', 'paymentsForRepurchaseOfPreferredStock', 'dividendPayoutPreferredStock',
    'proceedsFromIssuanceOfCommonStock', 'proceedsFromIssuanceOfPreferredStock',
    'proceedsFromSaleOfTreasuryStock', 'changeInExchangeRate', 'reportedCurrency_y',
    'investmentIncomeNet', 'reportedCurrency', 'otherNonCurrentAssets','commonStock', 
])


KeyError: "['surprise', 'surprisePercentage', 'reportedEPS', 'estimatedEPS', 'reportTime', 'estimatedPE', 'reportedPE', 'estimatedPrice', 'reportedPrice', 'Open_IBM', 'reportedCurrency_x', 'paymentsForOperatingActivities', 'proceedsFromOperatingActivities', 'changeInOperatingLiabilities', 'changeInOperatingAssets', 'changeInReceivables', 'changeInInventory', 'paymentsForRepurchaseOfPreferredStock', 'dividendPayoutPreferredStock', 'proceedsFromIssuanceOfCommonStock', 'proceedsFromIssuanceOfPreferredStock', 'proceedsFromSaleOfTreasuryStock', 'changeInExchangeRate', 'reportedCurrency_y', 'investmentIncomeNet', 'reportedCurrency', 'otherNonCurrentAssets'] not found in axis"

In [221]:
total_report = total_report.drop(columns=[
   'fiscalDateEnding','reportedDate'
])

In [230]:
# transform none to numer
total_report = total_report.replace(['None','NaN'],0)

# prepares total report to report
total_report = total_report.apply(pd.to_numeric)


## Feature Engineering Target Variable (ft. Fa)

In [308]:
compute_target = pd.DataFrame()

# takes difference between previous and current row
compute_target = total_report.diff()

# drops NaN first Row
compute_target = compute_target.iloc[1:].reset_index(drop=True)

compute_target

,operatingCashflow,depreciationDepletionAndAmortization,capitalExpenditures,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,dividendPayout,dividendPayoutCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromRepurchaseOfEquity,changeInCashAndCashEquivalents,netIncome_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Next_Day_IBM,priceDiscrepancy
0,1.136000e+09,-3.000000e+06,-6.400000e+07,-6.540000e+08,-7.967000e+09,8.438000e+09,-3.000000e+06,0.000000e+00,0.000000e+00,-13000000.0,-13000000.0,9.432000e+09,0.000000e+00,1.607000e+09,-6.560000e+08,-9.920000e+08,-1.223000e+09,-2.310000e+08,-3.100000e+07,-1.728000e+09,-4.700000e+07,-32000000.0,7.360000e+08,56000000.0,-31000000.0,-56000000.0,-2.560000e+08,-1.700000e+07,7000000.0,-10000000.0,-9.510000e+08,-2.950000e+08,-56000000.0,-6.470000e+08,-9.750000e+08,-1.007000e+09,-1.017000e+09,-6.560000e+08,1.424000e+09,1.524000e+09,-5.700000e+07,-5.700000e+07,102000000.0,6.258000e+09,-1.220000e+08,-9.900000e+07,2.720000e+08,2.170000e+08,4.090000e+08,-1.920000e+08,0.000000e+00,2.200000e+07,1.153000e+09,-2.300000e+08,2.022000e+09,-1.520000e+09,-4.000000e+06,6.870000e+08,-3.797000e+09,-1.898000e+09,3.542000e+09,1.350000e+08,1.662000e+09,-1.899000e+09,3.135000e+09,-8.310000e+08,-1.530000e+08,-4.000000e+06,-5.970000e+08,-3.700000e+07,-6.500000e+07,-2.880000e+08,-2355943.0,-3.947275,-1.175722
1,-1.873000e+09,8.900000e+07,1.700000e+07,-4.123000e+09,6.263000e+09,-4.995000e+09,2.330000e+08,0.000000e+00,0.000000e+00,-6000000.0,-6000000.0,-6.037000e+09,0.000000e+00,-6.050000e+08,-4.123000e+09,-7.900000e+07,-1.450000e+08,-6.600000e+07,2.700000e+07,1.215000e+09,-4.620000e+08,-44000000.0,-1.294000e+09,72000000.0,-117000000.0,-72000000.0,4.560000e+08,5.800000e+07,59000000.0,30000000.0,-5.534000e+09,-1.411000e+09,-72000000.0,-4.148000e+09,9.460000e+08,-5.606000e+09,-5.576000e+09,-4.123000e+09,-7.787000e+09,-6.983000e+09,-1.521000e+09,-1.521000e+09,191000000.0,-6.337000e+09,-8.110000e+08,-1.940000e+08,1.080000e+08,-1.913000e+09,6.200000e+07,-1.975000e+09,0.000000e+00,7.000000e+06,-6.304000e+09,-2.500000e+09,-6.261000e+09,-5.270000e+08,7.800000e+07,-2.506000e+09,1.910000e+09,1.050000e+09,-5.735000e+09,-1.600000e+07,-9.007000e+09,8.600000e+08,-8.884000e+09,-9.573000e+09,4.510000e+08,1.314000e+09,-1.528000e+09,-3.000000e+07,-6.420000e+08,-5.580000e+08,-3423169.0,-12.635883,-7.018845
2,-5.800000e+08,8.300000e+07,2.200000e+07,4.588000e+09,1.869000e+09,-4.909000e+09,-2.310000e+08,0.000000e+00,0.000000e+00,-3000000.0,-3000000.0,-3.077000e+09,0.000000e+00,-3.620000e+09,4.588000e+09,8.600000e+08,1.428000e+09,5.690000e+08,6.300000e+07,9.060000e+08,4.640000e+08,62000000.0,-4.600000e+07,-2000000.0,-25000000.0,2000000.0,7.700000e+07,1.630000e+08,34000000.0,49000000.0,6.132000e+09,1.544000e+09,2000000

In [310]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# maps values to signmod
compute_target = compute_target.applymap(sigmoid)
compute_target

#TODO: WEIGHTED VALUES????

C:\Users\alden\AppData\Local\Temp\ipykernel_7492\3613379759.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  compute_target = compute_target.applymap(sigmoid)


,operatingCashflow,depreciationDepletionAndAmortization,capitalExpenditures,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,dividendPayout,dividendPayoutCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromRepurchaseOfEquity,changeInCashAndCashEquivalents,netIncome_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Next_Day_IBM,priceDiscrepancy
0,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.622459,0.622459,0.500000,0.500000,0.731059,0.622459,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.731059,0.731059,0.731059,0.500000,0.622459,0.731059,0.731059,0.500000,0.731059,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.731059,0.731059,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.5,0.500000,0.504735,0.558684
1,0.500000,0.731059,0.731059,0.500000,0.731059,0.500000,0.731059,0.622459,0.622459,0.500000,0.500000,0.500000,0.622459,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.731059,0.500000,0.622459,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.5,0.500000,0.500001,0.500224
2,0.500000,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.622459,0.622459,0.500000,0.500000,0.500000,0.622459,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.500000,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.622459,0.731059,0.500000,0.622459,0.731059,0.731059,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.731059,0.731059,0.500000,0.731059,0.731059,0.5,0.500000,0.731059,0.605443
3,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.622459,0.622459,0.500000,0.500000,0.731059,0.622459,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0

In [ ]:
compute_target['SumOfNorm'] = compute_target.sum(axis=1)

,operatingCashflow,depreciationDepletionAndAmortization,capitalExpenditures,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,dividendPayout,dividendPayoutCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromRepurchaseOfEquity,changeInCashAndCashEquivalents,netIncome_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Next_Day_IBM,priceDiscrepancy,SumOfNorm
0,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.622459,0.622459,0.500000,0.500000,0.731059,0.622459,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.731059,0.731059,0.731059,0.500000,0.622459,0.731059,0.731059,0.500000,0.731059,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.731059,0.731059,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.5,0.500000,0.504735,0.558684,43.136545
1,0.500000,0.731059,0.731059,0.500000,0.731059,0.500000,0.731059,0.622459,0.622459,0.500000,0.500000,0.500000,0.622459,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.731059,0.500000,0.622459,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.500000,0.5,0.500000,0.500001,0.500224,43.073350
2,0.500000,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.622459,0.622459,0.500000,0.500000,0.500000,0.622459,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.500000,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.622459,0.731059,0.500000,0.622459,0.731059,0.731059,0.500000,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.731059,0.500000,0.500000,0.731059,0.731059,0.500000,0.731059,0.731059,0.5,0.500000,0.731059,0.605443,49.077493
3,0.731059,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.622459,0.622459,0.500000,0.500000,0.731059,0.622459,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.731059,0.731059,0.500000,0.500000,0.731059,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.731059,0.500000,0.500000,0.500000,0.500000,0.500000,0.7310

In [312]:
compute_target['FinRepScore'] = compute_target['SumOfNorm'] 

## Classification Model Training

In [315]:
total_report = total_report.iloc[1:].reset_index(drop=True)


In [336]:
X = total_report
y = compute_target["FinRepScore"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [371]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators':[5,10,20],
    'max_depth':[None,5,10],
    'min_samples_split':[2,5]
}


# Initialize and train the model
model = RandomForestRegressor()
grid_search = GridSearchCV(estimator=model,param_grid=param_grid,scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

# Best Params
print("Best Parameters:", grid_search.best_params_)
print("Best MSE:", -grid_search.best_score_)

Best Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 20}
Best MSE: 7.09968574429318


In [377]:
from sklearn.ensemble import RandomForestRegressor

# Insert best params
best = RandomForestRegressor(max_depth=10, min_samples_split=5,n_estimators=20)
best.fit(X_train,y_train)




RandomForestRegressor(max_depth=10, min_samples_split=5, n_estimators=20)

In [378]:
# Make predictions
y_pred = best.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE/Average Score:", mse/compute_target['FinRepScore'].mean())
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

MSE/Average Score: 0.1308210942240217
Mean Absolute Error (MAE): 1.7729202141902514
R² Score: -0.15515671937709863


# Export

In [ ]:
#exports model
import joblib
joblib.dump(model,'rf.joblib')

['rf.joblib']

In [ ]:
#loaded_model = joblib.load('rf.joblib')
#loaded.predict(X_Test)